Installing `meshplot`:

```
pip install pythreejs
pip install git+https://github.com/skoch9/meshplot.git
```

In [1]:
import meshplot as mp
import numpy as np
from collections import namedtuple
import gmsh

In [2]:
def load_gmsh_nodes(gmshpath, entity):
    '''
    Given a fullpath to some .msh file
    load in the mesh nodes IDs, triangles and coordinates.
    gmshpath -- path to gmsh file
    dimtag   -- tuple specifying the (dimensionality,tagID) being loaded
    If entity=(dim,tag) not provided then pull the first entity and return
    '''

    gmsh.initialize()
    gmsh.open(gmshpath)
    nodes, coords, params = gmsh.model.mesh.getNodes(entity[0], entity[1])
    coords = np.array(coords).reshape((len(coords) // 3, 3))
    gmsh.clear()

    return nodes - 1, coords, params


In [3]:
def load_gmsh_elems(gmshpath, entity):
    '''
    Wrapper function for loading gmsh elements
    '''

    gmsh.initialize()
    gmsh.open(gmshpath)
    nodes, elem_ids, node_maps = gmsh.model.mesh.getElements(
        entity[0], entity[1])
    gmsh.clear()

    return nodes, elem_ids[0] - 1, node_maps[0] - 1

In [4]:
def load_surf_verts(f_msh, entities):
    '''
    surf:       Path to gmsh MSH file
    entities:   List of entities to attempt of form (dim, tag)
    '''

    for dim, tag in entities:

        try:
            nodes, coords, params = load_gmsh_nodes(f_msh, (dim, tag))
        except ValueError:
            continue
        else:
            return nodes, coords, params

    logging.error("Could not properly load Mesh! Check entity tags!")
    raise ValueError

In [5]:
def load_surf_trigs(f_msh, entities):
    '''
    surf:       Path to gmsh MSH file
    entities:   List of entities to attempt of form (dim, tag)
    '''

    for dim, tag in entities:

        try:
            _, _, trigs = load_gmsh_elems(f_msh, (dim, tag))
        except ValueError:
            continue
        else:
            return trigs.reshape(-1, 3)

    logging.error("Could not properly load Mesh! Check entity tags!")
    raise ValueError

In [6]:
def decompose_gmsh(f_mesh, entity):
    '''
    Given a path to a mesh file load the:
        nodes
        coordinates
        triangles
    For the specified surface.
    Arguments:
        f_mesh          Path to GMSH mesh file
        entity          Tuple describing gmsh (dim, tag)
    '''

    # Load vertex data
    nodes, coords, _ = load_surf_verts(f_mesh, entity)
    trigs = load_surf_trigs(f_mesh, entity)

    # Re-normalize node/trig identities
    trigs = trigs - nodes.min()
    nodes = nodes - nodes.min()
    return SurfaceMesh(nodes, coords, trigs)


In [24]:
mp.website()
def gen_mshplot_html(brain, head, line, out_html):
    '''
    Generate a visualization of the centroid to head projection problem
    Arguments:
       brain:           SurfaceMesh of brain
       head:            SurfaceMesh of head
       texture:         Texture to apply to brain
       line:            DistResult object for line to visualize projection
    '''

    p = mp.plot(brain.coords, brain.triangles)

    p.add_mesh(head.coords, head.triangles, c=np.array([0.7, 0.7, 0.7]))
    p._Viewer__objects[1]['material'].transparent = True
    p._Viewer__objects[1]['material'].opacity = 0.5
    p._Viewer__objects[1]['material'].metalness = 0.1

    source_line = np.array([line.source])
    target_line = np.array([line.target])
    p.add_lines(source_line,
                target_line,
                shading={
                    "line_width": 20,
                    "line_color": "black"
                })

    p.save(out_html)

In [17]:
HEAD_ENTITIES = [(2, 1005)]
GM_ENTITIES = [(2, 1002)]
SurfaceMesh = namedtuple("SurfaceMesh", ["nodes", "coords", "triangles"])
Line = namedtuple("Line", ["source","target"])

In [20]:
f_msh = '../output/msn__1-0001_MagVenture_MC_B70.nii_scalar.msh'
dlpfc_scalp = np.array([-44.440907, 42.625675,  72.085838])
dlpfc_brain = np.array([-36.5493246,  37.43826707, 65.7456537 ])

In [21]:
head = decompose_gmsh(f_msh, HEAD_ENTITIES)
brain = decompose_gmsh(f_msh, GM_ENTITIES)
line = Line(dlpfc_scalp, dlpfc_brain)

In [26]:
gen_mshplot_html(brain, head, line, "test.html")

Plot saved to file test.html.
